In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_row', 50)

train = pd.read_csv('../data/train_ratings.csv')
test = pd.read_csv('../data/test_ratings.csv')
books = pd.read_csv('../data/books.csv')
users = pd.read_csv('../data/users.csv')

n = 0 # N개 이하 범주는 Others로, N은 유동적으로 할 것

# Books

In [74]:
books.info()

# 결측치가 있는 데이터(language, category, summary 결측치 메꾸기)
# 사용하지 않을 데이터 drop out

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             82343 non-null   object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [75]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg


In [76]:
# (이 변수들은 추후 사용 가능할수도 있으나 일단 지웁니다.)
books.drop(['book_title', 'summary', 'img_url', 'img_path'], axis = 1, inplace = True)
# 'book_title', 'summary'은 추후 텍스트 데이터로 사용
# 'img_url'은 isbn 검증 및 이미지 데이터로 사용 - path 대신 url 사용 시 시간이 너무 오래 걸리지는 않는지 확인

### Publisher

In [77]:
print(books['publisher'].nunique())

# books의 publisher 변수 중 이름이 비슷한 변수들을 찾아 하나로 통일해줍니다.
books_publishers = books.groupby('publisher')['isbn'].count().sort_values(ascending=False)
for i in books_publishers[books_publishers > 15].index: # size limit 문제 발생...
    books['publisher'][books['publisher'].str.contains(i)] = i

print(books['publisher'].nunique())

11571
10182


### Category

In [78]:
# 대괄호 써있는 카테고리 전치리
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
# 모두 소문자로 통일
books['category'] = books['category'].str.lower()

# 수작업으로 high 카테고리로 통합
categories = ['garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

books['category_high'] = books['category'].copy()
for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category_high'] = category

# category_high NULL 값을 최빈값으로 채웁니다.
# 근거 : category_high == fiction 일 때와 값이 NULL 일 때 평균이 유사
books['category_high'].fillna('fiction', inplace = True)

### language

In [79]:
# language와 NULL 값을 최빈값으로 채웁니다.
# 근거 : language == en일 때와 값이 NULL 일 때 rating 평균이 유사
books['language'].fillna('en', inplace = True)

### Years

In [80]:
# 출판연도 1970, 1980, 1990, 2000, 2020 으로 범주화 시킵니다.
# 딥러닝 과정에서 범주화 시키는 것이 유리합니다.
# 근거 : develop 파일에서 여러번 실험 결과 본 기준이 가장 rating을 잘 구분함.

books['years'] = books['year_of_publication'].copy()
books['years'][books['year_of_publication'] < 1970] = 1970
books['years'][(books['year_of_publication'] < 1980) * (books['year_of_publication'] >= 1970)] = 1980
books['years'][(books['year_of_publication'] < 1990) * (books['year_of_publication'] >= 1980)] = 1990
books['years'][(books['year_of_publication'] < 2000) * (books['year_of_publication'] >= 1990)] = 2000
books['years'][(books['year_of_publication'] >= 2000)] = 2020
books['years'] = books['years'].astype('int')
books['years'] = books['years'].astype('str')

In [81]:
# 사용한 변수를 제외하고 전처리가 잘 됬는지 확인합니다.
books.drop(['year_of_publication', 'category'], axis = 1, inplace = True)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn           149570 non-null  object
 1   book_author    149570 non-null  object
 2   publisher      149570 non-null  object
 3   language       149570 non-null  object
 4   category_high  149570 non-null  object
 5   years          149570 non-null  object
dtypes: object(6)
memory usage: 6.8+ MB


In [82]:
# 일정 개수(n) 이하 작가, 출판사, 언어, 카테고리 변수 모두 Others 취급

def make_others(_column):
    tem = pd.DataFrame(books[_column].value_counts()).reset_index()
    tem.columns = ['names','count']
    others_list = tem[tem['count'] < n]['names'].values  # n은 초기에 설정함. 바꿀 수 있음.
    books.loc[books[books[_column].isin(others_list)].index, _column]= 'others'

make_others('book_author')
make_others('publisher')
make_others('language')
make_others('category_high')

# Users

### Location

In [83]:
# location이 지역, 주, 국가로 되어있어 이 부분 기초 전처리 진행 과정입니다.

users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

# 지역, 주, 국가
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

# 도시는 존재하는데 나라 정보가 없는 경우 채워주는 코드
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

In [84]:
# 저는 도시, 주, 국가 중 주를 선택했습니다.
# 우선 모든 변수를 다 쓰는 건 아니라고 생각했어요. 도시 < 주 < 국가로 포함관계가 있기 때문이죠.
# 데이터 분석 결과 주와 국가 단위가 비슷한 경우가 많은 것을 확인했습니다.
# 조그만 섬 국가 < 미국 켈리포니아 주 < 미국 같은 경우죠.
# 미국 같은 경우 미국으로 뭉뚱그리기 보다 주 단위로 나누는 것이 맞다고 판단했습니다.
# 실제 미국 주 별로 rating 차이가 꽤 존재합니다.
# 다만 도시 기준으로 나누면 너무 세분화 될 것 같다는 생각이 들었습니다.
# 결론적으로 주를 지역을 나타내는 변수로 사용하기로 하고 결측값을 도시, 나라에서 채우기로 했습니다.

def _fillna(x):
    if pd.isna(x['location_country']):
        # 만약 나라가 기록 안되있는 경우        
        if pd.isna(x['location_city']):
            # 도시까지 없다면 모든 정보가 없음. 최빈값 california 사용.
            return 'california'
        else:
            tem = users['location_state'][users['location_city'] == x['location_city']].value_counts()
            if len(tem) == 0: 
                # 만약 주 이름이 없는 도시이면 도시 이름을 주 이름으로 사용.
                return x['location_city'] 
            else:
                # 그 도시에서 가장 자주 쓰이는 주 이름 사용.
                return tem.index[0]

    else:
        tem = users['location_state'][users['location_country'] == x['location_country']].value_counts()
        if len(tem) == 0: 
            # 만약 주 이름이 없는 나라이면 나라이름을 주 이름으로 사용.
            return x['location_country'] 
        else:
            # 그 나라에서 가장 자주 쓰이는 주 이름 사용.
            return tem.index[0]

users['fix_location_state'] = users.apply(lambda x : _fillna(x) if pd.isna(x['location_state']) else x['location_state'], axis = 1)

### Age

![image](https://user-images.githubusercontent.com/79916736/197685190-c4b88b82-4a23-4b79-b7a4-1fb849032dd3.png)

In [85]:
# 위 데이터를 바탕으로 나이를 분류합니다. 
# 이때 10세 미만과 나이가 NULL인 데이터는 상당히 유사한 평점을 메기는 것 같아요.
# 그러므로 10세 미만 사용자와 NULL 사용자는 같은 그룹으로 묶습니다. 

users['fix_age'] = users['age'].copy()
users['fix_age'][users['age'] < 10] = 10
users['fix_age'][(users['age'] < 20) & (users['age'] >= 10)] = 20
users['fix_age'][(users['age'] < 30) & (users['age'] >= 20)] = 30
users['fix_age'][(users['age'] < 35) & (users['age'] >= 30)] = 35
users['fix_age'][(users['age'] < 40) & (users['age'] >= 35)] = 40
users['fix_age'][(users['age'] < 50) & (users['age'] >= 40)] = 50
users['fix_age'][users['age'] >= 50] = 100
users['fix_age'].fillna(10, inplace = True)
users['fix_age'] = users['fix_age'].astype('int')
users['fix_age'] = users['fix_age'].astype('str')

In [86]:
# users에서 사용한 변수는 모두 제거합니다.
users = users[['user_id', 'fix_location_state', 'fix_age']]
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             68092 non-null  int64 
 1   fix_location_state  68092 non-null  object
 2   fix_age             68092 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [87]:
# 일정 개수(n) 이하 fix_location_state 범주 모두 Others 취급

def make_others2(_column):
    tem = pd.DataFrame(users[_column].value_counts()).reset_index()
    tem.columns = ['names','count']
    others_list = tem[tem['count'] < n]['names'].values  # n은 초기에 설정함. 바꿀 수 있음.
    users.loc[users[users[_column].isin(others_list)].index, _column]= 'others'

make_others2('fix_location_state')

In [88]:
# n값과 함께 csv 파일로 새로 저장함

books.to_csv(f"../data/mod_books_{n}n.csv", index = False)
users.to_csv(f"../data/mod_users_{n}n.csv", index = False)

In [89]:
# 전처리 완료한 books와 users 테이블을 이용해 rating 테이블과 merge 하기.

train_rating = pd.merge(train,books, how='right',on='isbn')
train_rating.dropna(subset=['rating'], inplace = True)
train_rating = pd.merge(train_rating, users, how='right',on='user_id')
train_rating.dropna(subset=['rating'], inplace = True)

test_rating = pd.merge(test,books, how='right',on='isbn')
test_rating.dropna(subset=['rating'], inplace = True)
test_rating = pd.merge(test_rating, users, how='right',on='user_id')
test_rating.dropna(subset=['rating'], inplace = True)


In [90]:
train_rating.to_csv(f"../data/mod_train_rating_{n}n.csv", index = False)
test_rating.to_csv(f"../data/mod_test_rating_{n}n.csv", index = False)

In [93]:
train_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 315083
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             306795 non-null  float64
 1   isbn                306795 non-null  object 
 2   rating              306795 non-null  float64
 3   book_author         306795 non-null  object 
 4   publisher           306795 non-null  object 
 5   language            306795 non-null  object 
 6   category_high       306795 non-null  object 
 7   years               306795 non-null  object 
 8   fix_location_state  306795 non-null  object 
 9   fix_age             306795 non-null  object 
dtypes: float64(2), object(8)
memory usage: 25.7+ MB
